In [1]:
import torch
from torch.utils.data import Dataset, random_split
import pandas as pd
import numpy as np
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import os.path
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
if torch.has_cuda:
    device = 'cuda'
elif torch.has_mps:
    device = 'mps'
else:
    device = 'cpu'

device = torch.device(device)
print(device)

mps


In [3]:
layer_size = 4312
model = nn.Sequential(
    nn.Linear(539, layer_size),
    nn.BatchNorm1d(layer_size),
    nn.Sigmoid(),
    nn.Linear(layer_size, layer_size),
    nn.BatchNorm1d(layer_size),
    nn.Sigmoid(),
    nn.Linear(layer_size, layer_size),
    nn.BatchNorm1d(layer_size),
    nn.Sigmoid(),
    nn.Linear(layer_size, layer_size),
    nn.BatchNorm1d(layer_size),
    nn.Sigmoid(),
    nn.Linear(layer_size, layer_size),
    nn.BatchNorm1d(layer_size),
    nn.Sigmoid(),
    nn.Linear(layer_size, layer_size),
    nn.BatchNorm1d(layer_size),
    nn.Sigmoid(),
    nn.Linear(layer_size, layer_size),
    nn.BatchNorm1d(layer_size),
    nn.Sigmoid(),
    nn.Linear(layer_size, 1)
)

num_parameters = sum(p.numel() for p in model.parameters())
print('Number of parameters: {}'.format(num_parameters))

Number of parameters: 113979097


In [4]:
model.load_state_dict(torch.load('./100_256.pth'))
model.to(device)
model.eval()

Sequential(
  (0): Linear(in_features=539, out_features=4312, bias=True)
  (1): BatchNorm1d(4312, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): Sigmoid()
  (3): Linear(in_features=4312, out_features=4312, bias=True)
  (4): BatchNorm1d(4312, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (5): Sigmoid()
  (6): Linear(in_features=4312, out_features=4312, bias=True)
  (7): BatchNorm1d(4312, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (8): Sigmoid()
  (9): Linear(in_features=4312, out_features=4312, bias=True)
  (10): BatchNorm1d(4312, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (11): Sigmoid()
  (12): Linear(in_features=4312, out_features=4312, bias=True)
  (13): BatchNorm1d(4312, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (14): Sigmoid()
  (15): Linear(in_features=4312, out_features=4312, bias=True)
  (16): BatchNorm1d(4312, eps=1e-05, momentum=0.1, affine=True, track_running_sta

In [5]:
test_data = pd.read_csv('./csv_artifacts/test_dataish.csv', dtype=np.float32)
trip_ids = pd.read_csv('./csv_artifacts/test_public.csv')
trip_ids = trip_ids[['TRIP_ID']]
all_xy = test_data.to_numpy()
test_input = torch.tensor(all_xy[:,0:539], dtype=torch.float32).to(device)

In [6]:
with torch.no_grad():
    outputs = model(test_input)

In [7]:
outputs = torch.Tensor.cpu(outputs)

In [8]:
output_file = pd.DataFrame(columns=['TRIP_ID', 'TRAVEL_TIME'])
display(output_file)

,TRIP_ID,TRAVEL_TIME


In [9]:
for count, i in enumerate(outputs):
    trip_id = trip_ids.iloc[count].to_string()
    trip_id = trip_id.replace('TRIP_ID', '').strip()
    print(trip_id)
    item = i.item()
    print('Trip ID: {} Result: {}'.format(count+1, item))
    output_file.loc[len(output_file)] = [trip_id, item]

T1
Trip ID: 1 Result: 824.544677734375
T2
Trip ID: 2 Result: 745.257080078125
T3
Trip ID: 3 Result: 743.2536010742188
T4
Trip ID: 4 Result: 607.1177368164062
T5
Trip ID: 5 Result: 723.1373901367188
T6
Trip ID: 6 Result: 1090.652099609375
T7
Trip ID: 7 Result: 712.9744262695312
T8
Trip ID: 8 Result: 729.9930419921875
T9
Trip ID: 9 Result: 815.5969848632812
T10
Trip ID: 10 Result: 769.829345703125
T11
Trip ID: 11 Result: 572.54833984375
T12
Trip ID: 12 Result: 949.3400268554688
T13
Trip ID: 13 Result: 636.6807250976562
T14
Trip ID: 14 Result: 950.2234497070312
T15
Trip ID: 15 Result: 805.1397094726562
T16
Trip ID: 16 Result: 1058.959716796875
T17
Trip ID: 17 Result: 670.947021484375
T18
Trip ID: 18 Result: 1044.61572265625
T19
Trip ID: 19 Result: 827.6145629882812
T20
Trip ID: 20 Result: 1048.875732421875
T21
Trip ID: 21 Result: 773.8358764648438
T22
Trip ID: 22 Result: 857.578369140625
T23
Trip ID: 23 Result: 614.93603515625
T24
Trip ID: 24 Result: 925.2547607421875
T25
Trip ID: 25 Resu

In [10]:
output_file.to_csv('submissions_5.csv', index=False)